In [37]:
import spacy
import pandas as pd
nlp = spacy.load("en_core_web_lg")
import dateparser
import pygwalker as pyg

In [39]:

#sentence = "send a reminder for the meeting at 5 pm to everyone now and create a document for meeting notes then send a message to Rajesh sir."
#sentence = "send me a reminder on mail regarding the meeting tomorrow"
sentence = "add a message for Ashwin to see Rajesh"
doc = nlp(sentence)
graph = {}


def generateTable(doc):

    token_text = []
    token_lemma = []
    token_pos = []
    token_tag = []
    token_dep = []
    token_children =[]
    times = []
    dates = []


    
    
    #get root_node 
    
    for token in doc:
        token_text.append(token.text)
        token_lemma.append(token.lemma_ )
        token_pos.append(token.pos_) 
        token_tag.append(token.tag_)
        token_dep.append(token.dep_)
        token_children.append([child.lemma_ for child in token.children])
        graph[token] = token.children
        if token.dep_=="ROOT":
            root_node = token

    df_dict = {
        "text":token_text,
        "lemma":token_lemma,
        "pos":token_pos,
        "dep":token_dep,
        "tag":token_tag,
        "children":token_children
    }   


#     get date time
    for ent in doc.ents:
            print(ent,ent.label_)
            if ent.label_ in ["TIME", "DATE"]:
                if ent.label_ == "TIME":
                    times.append(ent.text)
                elif ent.label_ == "DATE":
                    dates.append(ent.text)
    print(dates,times)

    df = pd.DataFrame(df_dict)

    return df,root_node

df , root_node = generateTable(doc)
df

Ashwin PERSON
Rajesh PERSON
[] []


,text,lemma,pos,dep,tag,children
0,add,add,VERB,ROOT,VB,"[message, see]"
1,a,a,DET,det,DT,[]
2,message,message,NOUN,dobj,NN,"[a, for]"
3,for,for,ADP,prep,IN,[Ashwin]
4,Ashwin,Ashwin,PROPN,pobj,NNP,[]
5,to,to,PART,aux,TO,[]
6,see,see,VERB,advcl,VB,"[to, Rajesh]"
7,Rajesh,Rajesh,PROPN,dobj,NNP,[]


Find the action in the sentence

In [11]:
functionality_space = {
    "add reminder":"ADD MESSAGE",
    "add message":"ADD MESSAGE",
    "tell":"ADD MESSAGE",
    "send message":"SEND MESSAGE",
    "drop message":"ADD MESSAGE",
    "have message":"GET MESSAGE",
    "remind":"ADD MESSAGE",
    "leave message":"ADD MESSAGE",
    "add":"ADD",
    "create list":"CREATE LIST",
    "add event":"ADD TO SCHEDULE",
    "schedule event":"ADD TO SCHEDULE",
    "add expense":"REMOVE FUND",
    "add fund":"ADD FUND",
    "remove fund":"REMOVE FUND",
    "spend":"REMOVE FUND"
    }



In [10]:
def find_functionality(action):
    best_ans = (0,0)

    for act in functionality_space:
        doc1 = nlp(act)
        doc2 = nlp(action)
        similarity = doc1.similarity(doc2)
        # print(data, similarity)
        if similarity > best_ans[0]:
            best_ans = (similarity,functionality_space[act])

    return best_ans


In [22]:
known_person = [
    "I",
    "everyone",
    "mashaal",
    "priyansh",
    "anmol",
    "divyansh",
    "rupesh",
    "rajesh",
    "ashutosh",
    "naveen",
    "harshit",
    "aswin",
    "herchelle",
]

In [8]:
known_objects = [
    "messsage",
    "schedule",
    "inventory",
    "funds",
    "list"
]

In [7]:
import difflib

def find_known_names(sentence, known_names):
    found_names = []
    for word in sentence.split():
        closest_match = difflib.get_close_matches(word, known_names, n=1, cutoff=0.7)
        if closest_match:
            found_names.append((word, closest_match[0]))
    return found_names

# found_names = find_known_names(sentence, known_person)
# print(found_names)

In [25]:
from transformers import pipeline
qa_model = pipeline("question-answering")


def break_querry(sentence):
    
    
    #return dict
    
    answer = {}
    
    #simplify the sentence and retrieve the root, action
    
    doc = nlp(sentence)
    simple_sentence = ""
    action = ""
    start = False
    
    for token in doc:
        if token.tag_ == "VBP":
            continue

        elif token.dep_=="ROOT":
            root_node = token
            start = True
            action+=" "+token.lemma_

            for t in token.children:

                if t.pos_=="NOUN" and t.dep_!="punct":
                    action+=" "+t.lemma_
                    got_action = True
                    break
        
        if start:
            simple_sentence+=" "+token.text


    sentence = simple_sentence.strip()
    action = action.strip()
    
    print("Simplified Sentence :",sentence)
    
    function =  find_functionality(action)
    if function[0]<0.6:
        phrase_handling(action)
        
    print("Functionlaity:",function)
    answer["FUNCTION"]=function
    
    print("Action :",action)
    answer["ACTION"]=action
    
    #find the target
    
    stack = [root_node]
    visited=[root_node]
    target_people = []
    while stack:
        token = stack.pop()
        
        for i in token.children:
            if i.pos_ not in ["VERB"] and i not in visited:
                stack.append(i)
                visited.append(i)
                if i.pos_ in ["PRON","PROPN"] :
                    target_people.append(i.lemma_)
                

    print("Who :",target_people)
    answer["WHO"]=target_people
    
    context = sentence
    
    
    #get what
    
    scheduled_for = " ".join(target_people)
    q1 = "What should I "+action +"for?"
    w1 = qa_model(question = q1, context = context)
    print("What:",w1['answer'])
    answer["WHAT"]=w1['answer']
    
    
    #get when parsed from date time

    dates = []
    times = []
    for ent in doc.ents:

        if ent.label_ in ["TIME", "DATE"]:
            if ent.label_ == "TIME":
                times.append(ent.text)
            elif ent.label_ == "DATE":
                dates.append(ent.text)

    date = "".join(dates)
    time = "".join(times)
    input_text = date + " " + time
    parsed_datetime = dateparser.parse(input_text)
    print("When:",parsed_datetime)
    answer["WHEN"]=parsed_datetime

    
    #get where

    q4 = "Where should I "+action 
    w4 = qa_model(question = q4, context = context)
    if w4["score"]>0.5:
        w4 = w4['answer']
    else:
        w4 = None

    print("Where:",w4)
    answer["WHERE"]=w4
    
    return answer
    
ans = break_querry(sentence)
print(ans)

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


Simplified Sentence : add a message for Ashwin to meet Herschelle
Functionlaity: (1.0, 'ADD MESSAGE')
Action : add message
Who : []
What: a message for Ashwin to meet Herschelle
When: None
Where: None
{'FUNCTION': (1.0, 'ADD MESSAGE'), 'ACTION': 'add message', 'WHO': [], 'WHAT': 'a message for Ashwin to meet Herschelle', 'WHEN': None, 'WHERE': None}
